# Fobidoshi

- Draw a circle in some empty white cells

- So that all circled cells form an orthogonally contiguous area and no horizontal or vertical stripe of more than 3 circled cells is created.

![](https://www.janko.at/Raetsel/Fobidoshi/Regeln-01.gif) ![](https://www.janko.at/Raetsel/Fobidoshi/Regeln-02.gif)

> Details on: https://www.janko.at/Raetsel/Fobidoshi/index.htm 

In [ ]:
import numpy as np
from ortools.sat.python import cp_model as cp
import gurobipy as gp
from collections import deque
from itertools import chain

In [ ]:
def readGrid(path):
    with open(f"../assets/data/Fobidoshi/{path}.txt") as f:
        num = f.readline()
        m, n = num.split(" ")[0], num.split(" ")[1]
        grid = f.readlines()
        res = [g.strip().split(" ") for g in grid]
        return int(m), int(n), res

if __name__ == "__main__":
    m, n, grid = readGrid("12x12_1")
    for g in grid:
        print(g)

In [ ]:
def FobidoshiSolver(m, n, grid) :
    """_summary_

    Args:
        m (_type_): 行数
        n (_type_): 列数
        grid (_type_): 网格
    """
    
    Fobidoshi = gp.Model("Fobidoshi")
    Fobidoshi.modelSense = gp.GRB.MAXIMIZE
    Fobidoshi.Params.lazyConstraints = 1
    Fobidoshi.update()
    x = {}
    for i in range(m):
        for j in range(n):
            x[i, j] = Fobidoshi.addVar(
            vtype = gp.GRB.BINARY,
            obj = 1,
            name = f"x[{i},{j}]")
            if grid[i][j] == "X":
                Fobidoshi.addConstr(x[i, j] == 1)
            elif grid[i][j] == "O":
                Fobidoshi.addConstr(x[i, j] == 0)
    for i in range(m):
        for j in range(n - 3):
            Fobidoshi.addConstr(gp.quicksum([x[i, j], x[i, j + 1], x[i, j + 2], x[i, j + 3]]) >= 1, name = f"x[{i},{j},r]")
    for j in range(n):
        for i in range(m - 3):
            Fobidoshi.addConstr(gp.quicksum([x[i, j], x[i + 1, j], x[i + 2, j], x[i + 3, j]]) >= 1, name = f"x[{i},{j},c]")
    
    for i in range(m):
        for j in range(n):
            neighbours = []
            directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
            for (subx, suby) in directions:
                if (i + subx >= 0 and i + subx < m) and (j + suby >= 0 and j + suby < n):
                    neighbours.append((i + subx, j + suby))
            Fobidoshi.addConstr(gp.quicksum(x[subx, suby] for (subx, suby) in neighbours) <= len(neighbours) - 1 + x[i, j], name = f"alone_{i}_{j}")
    
    Fobidoshi.setObjective(gp.quicksum(x[i, j] for i in range(m) for j in range(n)), gp.GRB.MAXIMIZE)
    # Fobidoshi.write("Fobidoshi.lp")
    def border_elim(model, where):
        if (where == gp.GRB.Callback.MIPSOL):
            x_sol = model.cbGetSolution(model._x)
            curr_grid = [[0] * n for _ in range(m)]
            for i in range(m):
                for j in range(n):
                    if x_sol[i, j] > 1e-6:
                        curr_grid[i][j] = 1
                    else:
                        curr_grid[i][j] = 0
            
            def find_surrounding_black_cells(grid):
                rows, cols = len(grid), len(grid[0])
                visited = set()
                directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right movements
                total_white =  rows * cols - sum(chain.from_iterable(grid))
                # for g in grid:
                #     print(g)
                # print("====")
                # print(f"total vacant is {total_white}")
                # Find the first unvisited white cell
                def find_unvisited_white():
                    for i in range(rows):
                        for j in range(cols):
                            if grid[i][j] == 0 and (i, j) not in visited:
                                return (i, j)
                    return None
                borders_cut = [] # record cells that formulate the border: list(tuple)
                cells_inside_borders = [] # record all cells inside the border: list(tuple)
                cnt_white = 0
                while len(visited) < total_white :
                    start = find_unvisited_white()
                    if not start:
                        break
                    # Use BFS to find all connected white cells and record the surrounding black cells
                    queue = deque([start])
                    surrounding_black_cells = set()  # Use a set to avoid duplicates
                    current_inside_cells = [] # record cells in current border
                    while queue:
                        x, y = queue.popleft()
                        if (x, y) in visited:
                            continue
                        visited.add((x, y))
                        current_inside_cells.append((x, y))
                        cnt_white += 1
                        # Check all four adjacent directions
                        for dx, dy in directions:
                            nx, ny = x + dx, y + dy
                            if 0 <= nx < rows and 0 <= ny < cols:
                                if grid[nx][ny] == 0 and (nx, ny) not in visited:
                                    queue.append((nx, ny))
                                elif grid[nx][ny] == 1:
                                    surrounding_black_cells.add((nx, ny))
                    borders_cut.append(list(surrounding_black_cells))
                    cells_inside_borders.append(current_inside_cells)
                    
                assert len(borders_cut) == len(cells_inside_borders)

                if len(visited) == sum(chain.from_iterable(grid)) and len(borders_cut) == 1:
                    
                    return [], []
                return borders_cut, cells_inside_borders
            
            borders_cut, cells_inside_borders = find_surrounding_black_cells(curr_grid)

            for borders, cells_inside in zip(borders_cut, cells_inside_borders):
                
                if len(cells_inside) < m * n // 2:
                # if len(borders) < m * n // 4:
                    for (cell_x, cell_y) in cells_inside:
                        # print("CUT ", end = " ")
                        # print(borders, (cell_x, cell_y))
                        model.cbLazy(gp.quicksum(model._x[subx, suby] for (subx, suby) in borders ) <= len(borders) - 1 + model._x[cell_x, cell_y])

    Fobidoshi._x = x
    Fobidoshi.optimize(border_elim)
    # Fobidoshi.computeIIS()
    # Fobidoshi.write("fobidoshi.ilp")
    # Visualize 
    for i in range(m):
        for j in range(n):
            if x[i, j].x > 1e-6:
                print(f"X", end = " ")
            else:
                print(f"O", end = " ")
        print()
    print()

    
if __name__ == "__main__":
    m, n, grid = readGrid("12x12_4")
    FobidoshiSolver(m, n, grid)
    